## Setup and Import Libraries

In [1]:
import os
import json
from typing import List
from langchain_core.documents import Document
from langchain_community.document_loaders import JSONLoader

In [2]:
os.makedirs("data/json_files", exist_ok=True)

## Saving JSON File

In [3]:
json_data = {
    "company": "TechCorp",
    "employees": [
        {
            "id": 1,
            "name": "John Doe",
            "role": "Software Engineer",
            "skills": ["Python", "JavaScript", "React"],
            "projects": [
                {"name": "RAG System", "status": "In Progress"},
                {"name": "Data Pipeline", "status": "Completed"}
            ]
        },
        {
            "id": 2,
            "name": "Jane Smith",
            "role": "Data Scientist",
            "skills": ["Python", "Machine Learning", "SQL"],
            "projects": [
                {"name": "ML Model", "status": "In Progress"},
                {"name": "Analytics Dashboard", "status": "Planning"}
            ]
        }
    ],
    "departments": {
        "engineering": {
            "head": "Mike Johnson",
            "budget": 1000000,
            "team_size": 25
        },
        "data_science": {
            "head": "Sarah Williams",
            "budget": 750000,
            "team_size": 15
        }
    }
}

json_data

{'company': 'TechCorp',
 'employees': [{'id': 1,
   'name': 'John Doe',
   'role': 'Software Engineer',
   'skills': ['Python', 'JavaScript', 'React'],
   'projects': [{'name': 'RAG System', 'status': 'In Progress'},
    {'name': 'Data Pipeline', 'status': 'Completed'}]},
  {'id': 2,
   'name': 'Jane Smith',
   'role': 'Data Scientist',
   'skills': ['Python', 'Machine Learning', 'SQL'],
   'projects': [{'name': 'ML Model', 'status': 'In Progress'},
    {'name': 'Analytics Dashboard', 'status': 'Planning'}]}],
 'departments': {'engineering': {'head': 'Mike Johnson',
   'budget': 1000000,
   'team_size': 25},
  'data_science': {'head': 'Sarah Williams',
   'budget': 750000,
   'team_size': 15}}}

In [4]:
with open(file="data/json_files/company_data.json", mode='w') as file:
    json.dump(json_data, file, indent=2)

In [5]:
jsonl_data = [
    {"timestamp": "2024-01-01", "event": "user_login", "user_id": 123},
    {"timestamp": "2024-01-01", "event": "page_view", "user_id": 123, "page": "/home"},
    {"timestamp": "2024-01-01", "event": "purchase", "user_id": 123, "amount": 99.99}
]

with open(file='data/json_files/events.jsonl', mode='w') as file:
    for item in jsonl_data:
        file.write(json.dumps(item) + '\n')

## JSON Data Processing Strategies

### Method 1: JSON Loader with JQ Schema

In [6]:
employee_loader = JSONLoader(
    file_path='data/json_files/company_data.json',
    jq_schema='.employees[]',  # jq query to extract each employee
    text_content=False  # Get full JSON objects
)

In [7]:
employee_docs = employee_loader.load()
print(f"Loaded {len(employee_docs)} employee documents")
print(f"First employee: {employee_docs[0].page_content[:200]}...")
print(employee_docs)

Loaded 2 employee documents
First employee: {"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status"...
[Document(metadata={'source': 'C:\\Users\\SAAD TARIQ\\github_repositories\\RAG-Bootcamp\\03-Data-Ingestion-and-Data-Parsing-Techniques\\data\\json_files\\company_data.json', 'seq_num': 1}, page_content='{"id": 1, "name": "John Doe", "role": "Software Engineer", "skills": ["Python", "JavaScript", "React"], "projects": [{"name": "RAG System", "status": "In Progress"}, {"name": "Data Pipeline", "status": "Completed"}]}'), Document(metadata={'source': 'C:\\Users\\SAAD TARIQ\\github_repositories\\RAG-Bootcamp\\03-Data-Ingestion-and-Data-Parsing-Techniques\\data\\json_files\\company_data.json', 'seq_num': 2}, page_content='{"id": 2, "name": "Jane Smith", "role": "Data Scientist", "skills": ["Python", "Machine Learning", "SQL"], "projects": [{"name"

### Method 2: Custom Processing

In [8]:
def process_json_intelligently(file_path: str) -> List[Document]:
    """
    Process JSON with intelligent flattening and context preservation
    """
    with open(file=file_path, mode='r') as file:
        data = json.load(file)
    
    documents = []
    
    # Strategy 1: Create documents for each employee with full context
    for emp in data.get('employees', []):
        content = f"""Employee Profile:
        Name: {emp['name']}
        Role: {emp['role']}
        Skills: {', '.join(emp['skills'])}

        Projects:"""
        for proj in emp.get('projects', []):
            content += f"\n- {proj['name']} (Status: {proj['status']})"
        
        doc = Document(
            page_content=content,
            metadata={
                'source': file_path,
                'data_type': 'employee_profile',
                'employee_id': emp['id'],
                'employee_name': emp['name'],
                'role': emp['role']
            }
        )
        documents.append(doc)

    return documents

In [9]:
process_json_intelligently(file_path="data/json_files/company_data.json")

[Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 1, 'employee_name': 'John Doe', 'role': 'Software Engineer'}, page_content='Employee Profile:\n        Name: John Doe\n        Role: Software Engineer\n        Skills: Python, JavaScript, React\n\n        Projects:\n- RAG System (Status: In Progress)\n- Data Pipeline (Status: Completed)'),
 Document(metadata={'source': 'data/json_files/company_data.json', 'data_type': 'employee_profile', 'employee_id': 2, 'employee_name': 'Jane Smith', 'role': 'Data Scientist'}, page_content='Employee Profile:\n        Name: Jane Smith\n        Role: Data Scientist\n        Skills: Python, Machine Learning, SQL\n\n        Projects:\n- ML Model (Status: In Progress)\n- Analytics Dashboard (Status: Planning)')]